In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical


IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 2
EPOCHS = 2
SAMPLES_PER_CLASS = 10


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train, y_test = y_train.flatten(), y_test.flatten()
x_train, x_test = x_train / 255.0, x_test / 255.0

mask_train = np.isin(y_train, [0, 1])
mask_test = np.isin(y_test, [0, 1])
x_train, y_train = x_train[mask_train], y_train[mask_train]
x_test, y_test = x_test[mask_test], y_test[mask_test]
y_train, y_test = to_categorical(y_train, NUM_CLASSES), to_categorical(y_test, NUM_CLASSES)

def downsample(x, y, n):
    indices = [np.random.choice(np.where(np.argmax(y, axis=1) == i)[0], n, replace=False) for i in range(NUM_CLASSES)]
    return np.concatenate([x[idx] for idx in indices]), np.concatenate([y[idx] for idx in indices])

x_train, y_train = downsample(x_train, y_train, SAMPLES_PER_CLASS)
x_test, y_test = downsample(x_test, y_test, SAMPLES_PER_CLASS)


def preprocess(img, lbl):
    img = tf.image.resize(img, IMAGE_SIZE)
    return img, lbl

train_ds = (tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))

test_ds = (tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))


base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

base_model.trainable = False
model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_ds, epochs=EPOCHS, validation_data=test_ds)


loss, accuracy = model.evaluate(test_ds)
print(f'Validation Loss: {loss:.4f}')
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


model.save('transfer_learning_model.h5')
print("Training completed and model saved.")


Epoch 1/2
1/1 ━━━━━━━━━━━━━━━━━━━━ 34s 34s/step - accuracy: 0.5000 - loss: 0.7324 - val_accuracy: 0.5000 - val_loss: 0.7045
Epoch 2/2
1/1 ━━━━━━━━━━━━━━━━━━━━ 42s 42s/step - accuracy: 0.5000 - loss: 0.7070 - val_accuracy: 0.5000 - val_loss: 0.6867
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.5000 - loss: 0.6867


Validation Loss: 0.6867
Validation Accuracy: 50.00%
Training completed and model saved.
